## Combining Demographic Data with Johns Hopkins COVID-19 Data

data was downloaded from https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data:
+ US county-level confirmed cases data: https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv
+ US county-level deaths data: https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv

In [63]:
import pandas as pd
from datetime import date
from datetime import datetime
import re

In [64]:
pd.set_option('display.expand_frame_repr', False) # the frame will be huge, don't expand
pd.set_option('display.precision', 4)

In [65]:
demo = pd.read_csv("counties.csv", dtype={'FIPS':float})
# looks like JH data doesn't have leading zeros in FIPS codes
confirmed = pd.read_csv("time_series_covid19_confirmed_US.txt")
deaths = pd.read_csv("time_series_covid19_deaths_US.txt")

In [66]:
demo.head()

,Unnamed: 0,household_size,empl_agriculture,empl_professional,empl_social,empl_services,empl_manufacturing,empl_retail,employed,prc_fam_poverty,...,population,pop_65_plus,health_ins,county,state,FIPS,area,domestic_passengers,intl_passengers,order started
0,"Morgan County, Alabama: Summary level: 050, st...",2.56,1.0792,11.1812,19.4094,8.3231,22.2135,10.2471,53742,9.9,...,119089,17.1838,98.8143,Morgan County,AL,1103.0,579.34,580000,0,04/04/20
1,"Kings County, California: Summary level: 050, ...",3.15,14.8108,7.4102,21.6017,8.9412,7.1271,9.3059,52644,15.6,...,151366,10.1826,90.0942,Kings County,CA,6031.0,1389.42,0,0,03/19/20
2,"Monterey County, California: Summary level: 05...",3.31,15.9900,10.0846,19.6731,10.8732,6.5126,8.9714,190707,10.5,...,435594,13.6574,96.2853,Monterey County,CA,6053.0,3280.60,186000,0,03/19/20
3,"Nevada County, California: Summary level: 050,...",2.37,1.3392,16.3689,20.6696,11.5335,3.5097,10.5988,44505,5.1,...,99696,27.8306,98.7723,Nevada County,CA,6057.0,957.77,0,0,03/19/20
4,"Shasta County, California: Summary level: 050,...",2.59,1.0668,9.3942,25.4620,11.4847,4.4179,12.8545,69649,9.5,...,180040,20.5660,99.1735,Shasta County,CA,6089.0,3775.40,0,0,03/19/20


In [67]:
confirmed.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20
0,16.0,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316.0,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,121,121,128,130,133,133,133,133,135,135
2,580.0,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,6,6,6,6,6,6,6,6,6,6
3,630.0,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,573,620,683,725,788,897,903,923,974,1043
4,850.0,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,43,45,45,50,51,51,51,51,51,51


In [68]:
last_date = confirmed.columns.values[-1]
last_date

'4/16/20'

In [69]:
# get date of at least 10 confirmed cases...
date_10_cases = list()

for x in range(confirmed.shape[0]):
    trans = confirmed.iloc[x].T
    trans = trans.iloc[11:] # just use the date fields
    trans = trans[trans >= 10]
    
    if len(trans) > 0:
        date_10_cases.append(trans.keys()[0])
    else:
        date_10_cases.append(last_date) # if county hasn't yet reached 10 cases, use the latest date in the data

In [70]:
confirmed["ten plus cases"] = date_10_cases

In [71]:
confirmed.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,ten plus cases
0,16.0,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,4/16/20
1,316.0,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,121,128,130,133,133,133,133,135,135,3/19/20
2,580.0,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,6,6,6,6,6,6,6,6,6,4/16/20
3,630.0,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,620,683,725,788,897,903,923,974,1043,3/20/20
4,850.0,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,45,45,50,51,51,51,51,51,51,3/24/20


In [72]:
confirmed[(confirmed["Province_State"] == "New York") & 
         ((confirmed["Admin2"] == "Bronx") | (confirmed["Admin2"] == "Kings") |
          (confirmed["Admin2"] == "New York") | (confirmed["Admin2"] == "Queens") |
          (confirmed["Admin2"] == "Richmond"))]
# looks like NYC counties (bronx, kings, queens, etc. all reported under New York County

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,ten plus cases
1835,8.4036e+07,US,USA,840,36005.0,Bronx,New York,US,40.8521,-73.8628,...,0,0,0,0,0,0,0,0,0,4/16/20
1856,8.4036e+07,US,USA,840,36047.0,Kings,New York,US,40.6362,-73.9494,...,0,0,0,0,0,0,0,0,0,4/16/20
1863,8.4036e+07,US,USA,840,36061.0,New York,New York,US,40.7673,-73.9715,...,81803,87028,92384,98308,103208,106763,110465,118302,123146,3/6/20
1873,8.4036e+07,US,USA,840,36081.0,Queens,New York,US,40.7109,-73.8168,...,0,0,0,0,0,0,0,0,0,4/16/20
1875,8.4036e+07,US,USA,840,36085.0,Richmond,New York,US,40.5858,-74.1481,...,0,0,0,0,0,0,0,0,0,4/16/20


In [73]:
deaths.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20
0,16.0,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0.0,0,0
1,316.0,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,4,4,4,4,5,5,5,5.0,5,5
2,580.0,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,1,1,1,1,1,1,1,1.0,1,1
3,630.0,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,23,24,33,39,42,44,45,45.0,51,56
4,850.0,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,1,1,1,1,1,1,1,1.0,1,1


In [74]:
deaths.shape

(3256, 98)

In [75]:
deaths.columns

Index(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Population',
       '1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20',
       '2/3/20', '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20',
       '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20',
       '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20',
       '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20',
       '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20', '3/5/20',
       '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20', '3/12/20',
       '3/13/20', '3/14/20', '3/15/20', '3/16/20', '3/17/20', '3/18/20',
       '3/19/20', '3/20/20', '3/21/20', '3/22/20', '3/23/20', '3/24/20',
       '3/25/20', '3/26/20', '3/27/20', '3/28/20', '3/29/20', '3/30/20',
       '3/31/20', '4/1/20', '4/2/20'

In [76]:
# nyc_counties = ["Bronx", "Kings", "New York", "Queens", "Richmond"]
deaths[(deaths["Province_State"] == "New York") & ((deaths["Admin2"] == "Bronx") | (deaths["Admin2"] == "Kings") |
                                                  (deaths["Admin2"] == "New York") | (deaths["Admin2"] == "Queens") |
                                                  (deaths["Admin2"] == "Richmond"))]
# it appears that all 5 counties in NYC are all being reported en masse as New York County
# SO will need to aggregate some data for NYC instead of leaving each of these counties individually in the data

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20
1835,8.4036e+07,US,USA,840,36005.0,Bronx,New York,US,40.8521,-73.8628,...,0,0,0,0,0,0,0,0.0,0,0
1856,8.4036e+07,US,USA,840,36047.0,Kings,New York,US,40.6362,-73.9494,...,0,0,0,0,0,0,0,0.0,0,0
1863,8.4036e+07,US,USA,840,36061.0,New York,New York,US,40.7673,-73.9715,...,4009,4571,5150,5820,6367,6898,7349,7905.0,8455,11477
1873,8.4036e+07,US,USA,840,36081.0,Queens,New York,US,40.7109,-73.8168,...,0,0,0,0,0,0,0,0.0,0,0
1875,8.4036e+07,US,USA,840,36085.0,Richmond,New York,US,40.5858,-74.1481,...,0,0,0,0,0,0,0,0.0,0,0


In [77]:
merged = pd.merge(demo, deaths, how='inner', on="FIPS", 
                  left_index=False, right_index=False)

In [78]:
merged.head()

,Unnamed: 0,household_size,empl_agriculture,empl_professional,empl_social,empl_services,empl_manufacturing,empl_retail,employed,prc_fam_poverty,...,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20
0,"Morgan County, Alabama: Summary level: 050, st...",2.56,1.0792,11.1812,19.4094,8.3231,22.2135,10.2471,53742,9.9,...,0,0,0,0,0,0,0,0.0,0,0
1,"Kings County, California: Summary level: 050, ...",3.15,14.8108,7.4102,21.6017,8.9412,7.1271,9.3059,52644,15.6,...,0,0,0,0,0,1,1,1.0,1,1
2,"Monterey County, California: Summary level: 05...",3.31,15.9900,10.0846,19.6731,10.8732,6.5126,8.9714,190707,10.5,...,2,2,2,3,3,3,3,3.0,3,3
3,"Nevada County, California: Summary level: 050,...",2.37,1.3392,16.3689,20.6696,11.5335,3.5097,10.5988,44505,5.1,...,1,1,1,1,1,1,1,1.0,1,1
4,"Shasta County, California: Summary level: 050,...",2.59,1.0668,9.3942,25.4620,11.4847,4.4179,12.8545,69649,9.5,...,3,3,3,3,3,3,3,3.0,3,3


In [79]:
merged.describe()

,household_size,empl_agriculture,empl_professional,empl_social,empl_services,empl_manufacturing,empl_retail,employed,prc_fam_poverty,avg_income,...,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20
count,827.0000,827.0000,827.0000,827.0000,827.0000,827.0000,827.0000,8.2700e+02,827.0000,827.0000,...,827.0000,827.0000,827.0000,827.0000,827.0000,827.0000,827.0000,827.0000,827.0000,827.0000
mean,2.5905,1.9294,10.1016,23.9089,9.6716,11.0223,11.5296,1.6397e+05,9.0312,32026.3688,...,14.2370,16.5212,18.5889,20.9166,23.3192,25.1778,26.8972,29.4389,32.4595,37.8875
std,0.2503,2.7136,3.5975,4.7552,2.7863,5.8550,2.0200,2.9310e+05,4.2493,7871.8697,...,143.3261,163.4332,184.3480,208.4848,228.2320,247.1663,263.5049,283.8116,304.0160,407.3928
min,1.9000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.7763e+04,1.3000,14605.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
25%,2.4100,0.5405,7.6825,20.9051,7.9680,6.7159,10.2838,4.4176e+04,5.9000,26839.5000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,1.0000,1.0000
50%,2.5500,1.0454,9.5856,23.2224,9.3013,10.0312,11.4295,7.5494e+04,8.4000,30614.0000,...,1.0000,2.0000,2.0000,2.0000,2.0000,2.0000,2.0000,3.0000,3.0000,3.0000
75%,2.7100,2.2877,11.9260,26.3873,10.8736,14.3158,12.6416,1.6863e+05,11.4000,35698.0000,...,5.0000,6.0000,7.0000,7.0000,8.0000,9.0000,9.0000,10.0000,11.0000,12.5000
max,4.1100,25.7236,30.1044,46.2286,31.9878,43.8800,21.3085,5.0014e+06,29.4000,74911.0000,...,4009.0000,4571.0000,5150.0000,5820.0000,6367.0000,6898.0000,7349.0000,7905.0000,8455.0000,11477.0000


In [80]:
merged.columns.values

array(['Unnamed: 0', 'household_size', 'empl_agriculture',
       'empl_professional', 'empl_social', 'empl_services',
       'empl_manufacturing', 'empl_retail', 'employed', 'prc_fam_poverty',
       'avg_income', 'prc_public_transp', 'population', 'pop_65_plus',
       'health_ins', 'county', 'state', 'FIPS', 'area',
       'domestic_passengers', 'intl_passengers', 'order started', 'UID',
       'iso2', 'iso3', 'code3', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Population',
       '1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20',
       '2/3/20', '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20',
       '2/9/20', '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20',
       '2/15/20', '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20',
       '2/21/20', '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20',
       '2/27/20', '2/28/20', '2/29/20', '3/1/2

In [81]:
merged.shape

(827, 119)

In [82]:
merged[["population", "Population"]] 
# ACS population and that used by JH data very close, though not exactly the same
# just use ACS population for consistency
# ultimately drop columns 'UID', 'iso2', 'iso3', 'code3', 'Admin2', 'Province_State',
#        'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Population', "Unnamed: 0"
# and drop/ignore for model "county", "state", "FIPS"

,population,Population
0,119089,119679
1,151366,152940
2,435594,434061
3,99696,99755
4,180040,180080
...,...,...
822,814901,822083
823,85129,84769
824,948201,945726
825,187365,187885


In [83]:
merged = merged.drop(['UID', 'iso2', 'iso3', 'code3', 'Admin2', 'Province_State','Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Population', "Unnamed: 0"], axis=1)

In [84]:
confirmed[["3/1/20", "3/15/20", "4/1/20"]].describe() 
# would have added 4/15/20 if data were from later in the month

,3/1/20,3/15/20,4/1/20
count,3256.0000,3256.0000,3256.0000
mean,0.0092,0.8962,65.5319
std,0.1934,9.9487,908.4749
min,0.0000,0.0000,0.0000
25%,0.0000,0.0000,0.0000
50%,0.0000,0.0000,2.0000
75%,0.0000,0.0000,11.0000
max,9.0000,387.0000,47439.0000


In [85]:
# add the date of 10+ confirmed deaths as a column
confirmed = confirmed[["FIPS", "3/1/20", "3/15/20", "4/1/20", "ten plus cases"]]
confirmed.columns = ["FIPS", "cases_march1", "cases_march15", "cases_april1", "ten plus cases"]

In [86]:
merged = pd.merge(merged, confirmed, how='inner', on="FIPS",
                 left_index=False, right_index=False)

In [87]:
merged.head()

,household_size,empl_agriculture,empl_professional,empl_social,empl_services,empl_manufacturing,empl_retail,employed,prc_fam_poverty,avg_income,...,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,cases_march1,cases_march15,cases_april1,ten plus cases
0,2.56,1.0792,11.1812,19.4094,8.3231,22.2135,10.2471,53742,9.9,27742,...,0,0,0,0.0,0,0,0,0,19,3/28/20
1,3.15,14.8108,7.4102,21.6017,8.9412,7.1271,9.3059,52644,15.6,22628,...,0,1,1,1.0,1,1,0,0,4,4/12/20
2,3.31,15.9900,10.0846,19.6731,10.8732,6.5126,8.9714,190707,10.5,30674,...,3,3,3,3.0,3,3,0,0,42,3/21/20
3,2.37,1.3392,16.3689,20.6696,11.5335,3.5097,10.5988,44505,5.1,37645,...,1,1,1,1.0,1,1,0,0,26,3/28/20
4,2.59,1.0668,9.3942,25.4620,11.4847,4.4179,12.8545,69649,9.5,28144,...,3,3,3,3.0,3,3,0,1,7,4/3/20


In [88]:
merged.shape

(827, 111)

Combining the data for the 5 NYC counties since infections and data is only being reported through New York County instead of each county (New York County, Bronx County, Kings County, Queens County, Richmond County) individually

In [89]:
# nyc_counties = ["Bronx", "Kings", "New York", "Queens", "Richmond"]
deaths[(deaths["Province_State"] == "New York") & ((deaths["Admin2"] == "Bronx") | (deaths["Admin2"] == "Kings") |
                                                  (deaths["Admin2"] == "New York") | (deaths["Admin2"] == "Queens") |
                                                  (deaths["Admin2"] == "Richmond"))]
# it appears that all 5 counties in NYC are all being reported en masse as New York County
# SO will need to aggregate some data for NYC instead of leaving each of these counties individually in the data

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20
1835,8.4036e+07,US,USA,840,36005.0,Bronx,New York,US,40.8521,-73.8628,...,0,0,0,0,0,0,0,0.0,0,0
1856,8.4036e+07,US,USA,840,36047.0,Kings,New York,US,40.6362,-73.9494,...,0,0,0,0,0,0,0,0.0,0,0
1863,8.4036e+07,US,USA,840,36061.0,New York,New York,US,40.7673,-73.9715,...,4009,4571,5150,5820,6367,6898,7349,7905.0,8455,11477
1873,8.4036e+07,US,USA,840,36081.0,Queens,New York,US,40.7109,-73.8168,...,0,0,0,0,0,0,0,0.0,0,0
1875,8.4036e+07,US,USA,840,36085.0,Richmond,New York,US,40.5858,-74.1481,...,0,0,0,0,0,0,0,0.0,0,0


In [46]:
merged[(merged["state"] == "NY") & ((merged["county"] == "Bronx County") | 
                                   (merged["county"] == "Kings County") |
                                   (merged["county"] == "New York County") | 
                                   (merged["county"] == "Queens County") |
                                   (merged["county"] == "Richmond County"))][["county", "4/16/20", "ten plus cases", 
                                                                              "population", "domestic_passengers"]]
# combine Bronx, NY, Kings, Richmond, and Queens County into NY County entry

,county,4/16/20,ten plus cases,population,domestic_passengers
284,Bronx County,0,4/16/20,1432132,44647000
489,New York County,11477,3/6/20,1628701,44647000
588,Kings County,0,4/16/20,2582830,43872000
688,Queens County,0,4/16/20,2278906,44647000
689,Richmond County,0,4/16/20,476179,43872000


In [90]:
subset = merged[(merged["state"] == "NY") & ((merged["county"] == "Bronx County") | 
                                   (merged["county"] == "Kings County") |
                                   (merged["county"] == "New York County") | 
                                   (merged["county"] == "Queens County") |
                                   (merged["county"] == "Richmond County"))]
subset

,household_size,empl_agriculture,empl_professional,empl_social,empl_services,empl_manufacturing,empl_retail,employed,prc_fam_poverty,avg_income,...,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,cases_march1,cases_march15,cases_april1,ten plus cases
284,2.74,0.1590,9.2257,32.4218,11.8365,3.3429,11.2092,599204,24.3,21712,...,0,0,0,0.0,0,0,0,0,0,4/16/20
489,2.08,0.1234,21.6381,23.1955,10.3561,2.5411,6.4669,901880,12.1,74911,...,6367,6898,7349,7905.0,8455,11477,0,269,47439,3/6/20
588,2.62,0.0763,14.7999,28.0411,10.1623,3.1124,8.6798,1237382,15.4,34709,...,0,0,0,0.0,0,0,0,0,0,4/16/20
688,2.86,0.0308,11.5763,24.1851,11.7075,3.1511,9.3785,1139938,9.1,31866,...,0,0,0,0.0,0,0,0,0,0,4/16/20
689,2.80,0.0608,11.6203,29.4618,6.8848,2.6954,9.3508,215330,8.9,35966,...,0,0,0,0.0,0,0,0,0,0,4/16/20


In [91]:
subset.columns.values
# combine data for all fields up to the dates (deaths by date), and from then on use the values for NY County only

array(['household_size', 'empl_agriculture', 'empl_professional',
       'empl_social', 'empl_services', 'empl_manufacturing',
       'empl_retail', 'employed', 'prc_fam_poverty', 'avg_income',
       'prc_public_transp', 'population', 'pop_65_plus', 'health_ins',
       'county', 'state', 'FIPS', 'area', 'domestic_passengers',
       'intl_passengers', 'order started', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20',
       '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20', '2/4/20',
       '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20', '2/10/20',
       '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20', '2/16/20',
       '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20', '2/22/20',
       '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20', '2/28/20',
       '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20', '3/5/20',
       '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20',
       '3/12/20', '3/13/20', '3/14/20',

In [92]:
all_pop = sum(subset["population"])

In [93]:
nyc = list()
# use weighted averages except for employed,  population, area, and dom. and intl. passengers

for field in ['household_size', 'empl_agriculture', 'empl_professional','empl_social', 
              'empl_services', 'empl_manufacturing','empl_retail']:
    value = 0
    for x in range(subset.shape[0]):
        value = value + subset.iloc[x]["population"]/all_pop * subset.iloc[x][field]
    nyc.append(value)

In [94]:
nyc.append(sum(subset.employed))

In [95]:
for field in ['prc_fam_poverty', 'avg_income', 'prc_public_transp']:
    value = 0
    for x in range(subset.shape[0]):
        value = value + subset.iloc[x]["population"]/all_pop * subset.iloc[x][field]
    nyc.append(value)

In [96]:
nyc.append(sum(subset.population))

In [97]:
for field in ['pop_65_plus', 'health_ins']:
    value = 0
    for x in range(subset.shape[0]):
        value = value + subset.iloc[x]["population"]/all_pop * subset.iloc[x][field]
    nyc.append(value)

In [98]:
nyc.append(subset[subset["county"] == "New York County"].county.values[0])
nyc.append(subset[subset["county"] == "New York County"].state.values[0])
nyc.append(subset[subset["county"] == "New York County"].FIPS.values[0])

In [99]:
nyc.append(sum(subset.area))

In [100]:
nyc.append(max(subset.domestic_passengers))
nyc.append(max(subset.intl_passengers))

In [101]:
subset["order started"] # same for all 5 counties

284    03/22/20
489    03/22/20
588    03/22/20
688    03/22/20
689    03/22/20
Name: order started, dtype: object

In [102]:
for field in ['order started', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20',
       '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20', '2/4/20',
       '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20', '2/10/20',
       '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20', '2/16/20',
       '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20', '2/22/20',
       '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20', '2/28/20',
       '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20', '3/5/20',
       '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20',
       '3/12/20', '3/13/20', '3/14/20', '3/15/20', '3/16/20', '3/17/20',
       '3/18/20', '3/19/20', '3/20/20', '3/21/20', '3/22/20', '3/23/20',
       '3/24/20', '3/25/20', '3/26/20', '3/27/20', '3/28/20', '3/29/20',
       '3/30/20', '3/31/20', '4/1/20', '4/2/20', '4/3/20', '4/4/20',
       '4/5/20', '4/6/20', '4/7/20', '4/8/20', '4/9/20', '4/10/20',
       '4/11/20', '4/12/20', '4/13/20', '4/14/20', '4/15/20', '4/16/20']:
    nyc.append(subset[subset["county"] == "New York County"][field].values[0])

In [103]:
subset[["county",'cases_march1', 'cases_march15', 'cases_april1', "ten plus cases"]] 
# use the New York County values

,county,cases_march1,cases_march15,cases_april1,ten plus cases
284,Bronx County,0,0,0,4/16/20
489,New York County,0,269,47439,3/6/20
588,Kings County,0,0,0,4/16/20
688,Queens County,0,0,0,4/16/20
689,Richmond County,0,0,0,4/16/20


In [104]:
nyc.append(subset[subset["county"] == "New York County"]["cases_march1"].values[0])
nyc.append(subset[subset["county"] == "New York County"]["cases_march15"].values[0])
nyc.append(subset[subset["county"] == "New York County"]["cases_april1"].values[0])
nyc.append(subset[subset["county"] == "New York County"]["ten plus cases"].values[0])

In [105]:
merged.shape[1] == len(nyc)

True

In [106]:
subset["county"] # drop rows at indices 284, 489, 588, 688, 689 and then concat the new NYC row

284       Bronx County
489    New York County
588       Kings County
688      Queens County
689    Richmond County
Name: county, dtype: object

In [107]:
merged.iloc[merged.index[[284, 489, 588, 688, 689]]][["county", "state", "domestic_passengers","4/16/20"]]

,county,state,domestic_passengers,4/16/20
284,Bronx County,NY,44647000,0
489,New York County,NY,44647000,11477
588,Kings County,NY,43872000,0
688,Queens County,NY,44647000,0
689,Richmond County,NY,43872000,0


In [108]:
print(merged.shape)
merged = merged.drop(merged.index[[284, 489, 588, 688, 689]])
print(merged.shape)

(827, 111)
(822, 111)


In [109]:
nyc_df = pd.DataFrame([nyc], columns=merged.columns)

In [110]:
final = merged.append(nyc_df)

In [111]:
final.shape

(823, 111)

In [112]:
final.tail()

,household_size,empl_agriculture,empl_professional,empl_social,empl_services,empl_manufacturing,empl_retail,employed,prc_fam_poverty,avg_income,...,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,cases_march1,cases_march15,cases_april1,ten plus cases
823,2.4600,2.0402,6.4981,25.5142,7.5528,19.9115,11.0063,47691,6.4000,33545.0000,...,0,0,0,0.0,0,0,0,0,12,3/30/20
824,2.4100,0.3157,10.7249,26.2953,9.1475,14.7681,11.1703,461177,14.1000,28641.0000,...,85,88,94,101.0,105,114,0,6,780,3/16/20
825,2.4700,1.3699,8.8880,20.7820,8.2777,22.2426,10.2366,99134,5.4000,32489.0000,...,2,2,2,2.0,2,2,0,0,12,3/31/20
826,2.4800,0.5250,8.6500,23.3677,8.3442,21.1559,11.4266,95803,8.5000,29897.0000,...,3,3,4,5.0,6,6,0,1,28,3/28/20
0,2.6111,0.0863,14.1205,26.8827,10.7188,3.0278,8.9096,4093734,14.1997,39588.6851,...,6367,6898,7349,7905.0,8455,11477,0,269,47439,3/6/20


In [113]:
final.to_csv("combined_data.csv")